## Environment setup & installations

In [ ]:
# Create a conda environemnt
# conda create langchain python==3.10
# conda activate langchain

In [ ]:
# Langchain Installation
!pip install langchain

In [ ]:
!pip list | grep langchain

In [ ]:
!pip install langchain-openai

In [ ]:
!pip install faiss-cpu

In [ ]:
!pip install beautifulsoup4

In [ ]:
pip install langchainhub

In [43]:
!pip install numexpr

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.7/91.7 kB 3.8 MB/s eta 0:00:00


# LLM Chain

In [85]:
import os
# Set your keys here!
os.environ['OPENAI_API_KEY']="Give Your API Key"
# Get your key from here: https://tavily.com/ (Used for Agents)
# Tavily’s Search API is a search engine built specifically for AI agents (LLMs), 
# delivering real-time, accurate, and factual results at speed.
os.environ['TAVILY_API_KEY'] = 'Give Your TAVILY_API_KEY Key'

In [86]:
# using OpenAI (a popular model available via API)
from langchain_openai import ChatOpenAI
llm = ChatOpenAI()

In [87]:
llm.invoke("What is langsmith?")

AIMessage(content='Langsmith is not a widely known term or concept. It does not appear to have a specific definition or meaning. It is possible that it could be a proper noun, such as a name of a person or a company, but without further context, it is difficult to determine its specific reference.')

In [88]:
# We can guide it's response with a prompt template. 
# Prompt templates are used to convert raw user input to a better input to the LLM
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are world class technical documentation writer. Be as true as possible"),
    ("user", "{input}")
])

In [89]:
# combine these into a simple LLM chain
chain = prompt | llm

In [90]:
# invoke it and ask the same question. 
# It still won't know the answer, but it should respond in a more proper tone for a technical writer!
chain.invoke({"input": "What is LangSmith?"})

AIMessage(content='LangSmith is a powerful programming language designed to be simple, efficient, and versatile. It was developed with the goal of providing programmers with a comprehensive set of tools and features to facilitate the development of robust and scalable software applications.\n\nOne of the key features of LangSmith is its expressive syntax, which allows developers to write clean and readable code. The language supports a wide range of programming paradigms, including procedural, object-oriented, and functional programming, making it suitable for various application domains.\n\nLangSmith provides an extensive standard library, offering a rich set of functions and modules that can be used to streamline development tasks. It also supports seamless integration with other programming languages, allowing developers to leverage existing code and libraries.\n\nThe language is known for its powerful error handling mechanisms, which enable developers to write robust and reliable s

In [91]:
# simple output parser to convert the chat message to a string
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

In [92]:
chain = prompt | llm | output_parser

In [93]:
chain.invoke({"input": "What is LangSmith?"})

'LangSmith is a cutting-edge programming language that combines the power and flexibility of multiple programming paradigms to enable developers to write efficient and maintainable code. It is designed to meet the needs of modern software development by providing a rich set of features and tools for creating scalable and robust applications.\n\nWith LangSmith, developers can leverage object-oriented, functional, and procedural programming styles, allowing them to choose the approach that best suits their specific requirements. The language supports static typing, which helps catch errors early in the development process and improves code quality.\n\nOne of the key features of LangSmith is its extensive standard library, which provides a wide range of built-in functions and modules for common tasks such as file I/O, networking, and data manipulation. This library empowers developers to streamline their development process by reducing the need for external dependencies.\n\nLangSmith also

## Retrieval Chain

In [94]:
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://docs.smith.langchain.com/overview")

docs = loader.load()

In [95]:
from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()

In [96]:
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)

In [97]:
from langchain.chains import create_retrieval_chain
retriever = vector.as_retriever()

In [98]:
from langchain.chains.combine_documents import create_stuff_documents_chain

prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:
<context>
{context}
</context>
Question: {input}""")

document_chain = create_stuff_documents_chain(llm, prompt)

In [99]:
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [100]:
response = retrieval_chain.invoke({"input": "What is LangSmith?"})
print(response["answer"])

LangSmith is a tool or platform used for building reliable LLM (Language Learning Model) applications. It provides features such as tracing, debugging, monitoring, testing, evaluation, and dataset exporting. It helps developers understand and optimize the performance of prompts, chains, and agents in LLM applications.


## Conversation Retrieval Chain

In [101]:
# for observability
from langchain.globals import set_verbose, set_debug
set_verbose(True)
# Set this to true to see how Conversation Retrieval Chain works!
set_debug(False)

In [102]:
from langchain_core.messages import HumanMessage, AIMessage
chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]

In [103]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

# First we need a prompt that we can pass into an LLM to generate this search query
prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    ("user", "Given the above conversation, generate a search query to look up in order to get information relevant to the conversation")
])
retriever_chain = create_history_aware_retriever(llm, retriever, prompt)

In [104]:
retriever_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})

[Document(page_content='LangSmith Overview and User Guide | 🦜️🛠️ LangSmith', metadata={'source': 'https://docs.smith.langchain.com/overview', 'title': 'LangSmith Overview and User Guide | 🦜️🛠️ LangSmith', 'description': 'Building reliable LLM applications can be challenging. LangChain simplifies the initial setup, but there is still work needed to bring the performance of prompts, chains and agents up the level where they are reliable enough to be used in production.', 'language': 'en'}),
 Document(page_content="Skip to main content🦜️🛠️ LangSmith DocsPython DocsJS/TS DocsSearchGo to AppLangSmithOverviewTracingTesting & EvaluationOrganizationsHubLangSmith CookbookOverviewOn this pageLangSmith Overview and User GuideBuilding reliable LLM applications can be challenging. LangChain simplifies the initial setup, but there is still work needed to bring the performance of prompts, chains and agents up the level where they are reliable enough to be used in production.Over the past two months, 

In [105]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer the user's questions based on the below context:\n\n{context}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
])
document_chain = create_stuff_documents_chain(llm, prompt)

retrieval_chain = create_retrieval_chain(retriever_chain, document_chain)

In [106]:
chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]
retrieval_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})

{'chat_history': [HumanMessage(content='Can LangSmith help test my LLM applications?'),
  AIMessage(content='Yes!')],
 'input': 'Tell me how',
 'context': [Document(page_content="Skip to main content🦜️🛠️ LangSmith DocsPython DocsJS/TS DocsSearchGo to AppLangSmithOverviewTracingTesting & EvaluationOrganizationsHubLangSmith CookbookOverviewOn this pageLangSmith Overview and User GuideBuilding reliable LLM applications can be challenging. LangChain simplifies the initial setup, but there is still work needed to bring the performance of prompts, chains and agents up the level where they are reliable enough to be used in production.Over the past two months, we at LangChain have been building and using LangSmith with the goal of bridging this gap. This is our tactical user guide to outline effective ways to use LangSmith and maximize its benefits.On by default\u200bAt LangChain, all of us have LangSmith’s tracing running in the background by default. On the Python side, this is achieved by s

## Agent

In [107]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever,
    "langsmith_search",
    "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!",
)

In [108]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults()

In [109]:
tools = [retriever_tool, search]

In [110]:
from langchain_openai import ChatOpenAI
from langchain import hub
from langchain.agents import create_openai_functions_agent
from langchain.agents import AgentExecutor

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
agent = create_openai_functions_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [111]:
agent_executor.invoke({"input": "how can langsmith help with testing?"})



> Entering new AgentExecutor chain...
LangSmith can help with testing in several ways:

1. Test Case Generation: LangSmith can generate test cases automatically based on the specifications or requirements of the software being tested. It uses advanced techniques such as symbolic execution and constraint solving to generate test inputs that cover different paths and conditions in the code.

2. Test Oracles: LangSmith can help in defining test oracles, which are the expected outputs or behaviors of the software under test. It can analyze the code and specifications to automatically generate assertions or checks that can be used as test oracles.

3. Test Coverage Analysis: LangSmith can analyze the code and track the coverage of the test cases. It can provide information on which parts of the code are covered by the tests and which parts are not. This helps in identifying areas that need more testing and improving the overall test coverage.

4. Test Prioritization: LangSmith can priorit

{'input': 'how can langsmith help with testing?',
 'output': 'LangSmith can help with testing in several ways:\n\n1. Test Case Generation: LangSmith can generate test cases automatically based on the specifications or requirements of the software being tested. It uses advanced techniques such as symbolic execution and constraint solving to generate test inputs that cover different paths and conditions in the code.\n\n2. Test Oracles: LangSmith can help in defining test oracles, which are the expected outputs or behaviors of the software under test. It can analyze the code and specifications to automatically generate assertions or checks that can be used as test oracles.\n\n3. Test Coverage Analysis: LangSmith can analyze the code and track the coverage of the test cases. It can provide information on which parts of the code are covered by the tests and which parts are not. This helps in identifying areas that need more testing and improving the overall test coverage.\n\n4. Test Priorit

In [112]:
agent_executor.invoke({"input": "what is the weather in Bangalore?"})



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'weather in Bangalore'}`


[{'url': 'https://weatherspark.com/h/m/108998/2024/1/Historical-Weather-in-January-2024-in-Bengaluru-India', 'content': 'January 2024 Weather History in Bengaluru India  Bengaluru Temperature History January 2024 Hourly Temperature in January 2024 in Bengaluru  Observed Weather in January 2024 in Bengaluru Hours of Daylight and Twilight in January 2024 in Bengaluru  Compare Bengaluru to another city: Cloud Cover in January 2024 in BengaluruPartly Cloudy 1,500 ft. This report shows the past weather for Bengaluru, providing a weather history for January 2024. It features all historical weather data series we have available, including the Bengaluru temperature history for January 2024. You can drill down from year to month and even day level reports by clicking on the graphs.'}, {'url': 'https://www.weather2travel.com/india/bangalore/january/', 'content': 'Bangalore w

{'input': 'what is the weather in Bangalore?',
 'output': 'I found some information about the weather in Bangalore. In January 2024, the average maximum temperature is expected to be around 27°C during the day, with a minimum night temperature of 15°C. There will be about 9 hours of sunshine per day and a low chance of rainfall, with an average of 3 mm of monthly rainfall. The weather is generally characterized as having low heat and humidity. Please note that this information is based on historical weather data and may vary.'}

# ReAct

In [113]:
from langchain import hub
from langchain.agents import AgentExecutor, create_react_agent
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain.chains import LLMMathChain
from langchain_openai import OpenAI
from langchain.tools import tool

In [114]:
# Chain that interprets a prompt and executes python code to do math.
llm_math = LLMMathChain.from_llm(OpenAI())

In [115]:
@tool
def calculator(query: str) -> str:
    """A calculator for performing math."""
    return llm_math.invoke(query)

In [116]:
# Tavily’s Search API is a search engine built specifically for AI agents (LLMs)
search = TavilySearchResults(max_results=1)

In [117]:
tools = [search, calculator]

In [118]:
# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/react")
prompt

PromptTemplate(input_variables=['agent_scratchpad', 'input', 'tool_names', 'tools'], template='Answer the following questions as best you can. You have access to the following tools:\n\n{tools}\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [{tool_names}]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin!\n\nQuestion: {input}\nThought:{agent_scratchpad}')

In [119]:
# Choose the LLM to use
llm = OpenAI()

# Construct the ReAct agent
agent = create_react_agent(llm, tools, prompt)

In [120]:
# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [124]:
agent_executor.invoke({"input": "What is the weather in Bangalore? Convert it to Celsius."})



> Entering new AgentExecutor chain...
 I should use tavily_search_results_json to find the current weather in Bangalore
Action: tavily_search_results_json
Action Input: "weather in Bangalore"[{'url': 'https://weatherspark.com/h/m/108998/2024/1/Historical-Weather-in-January-2024-in-Bengaluru-India', 'content': 'January 2024 Weather History in Bengaluru India  Bengaluru Temperature History January 2024 Hourly Temperature in January 2024 in Bengaluru  Observed Weather in January 2024 in Bengaluru Hours of Daylight and Twilight in January 2024 in Bengaluru  Compare Bengaluru to another city: Cloud Cover in January 2024 in BengaluruPartly Cloudy 1,500 ft. This report shows the past weather for Bengaluru, providing a weather history for January 2024. It features all historical weather data series we have available, including the Bengaluru temperature history for January 2024. You can drill down from year to month and even day level reports by clicking on the graphs.'}] I should use calcula

{'input': 'What is the weather in Bangalore? Convert it to Celsius.',
 'output': '0 degrees Celsius'}

In [122]:
agent_executor.invoke({"input": "Who is Olivia Wilde's boyfriend? What is his current age raised to the 0.23 power?"})



> Entering new AgentExecutor chain...
 I should use tavily_search_results_json because I need accurate and trusted results for current events.
Action: tavily_search_results_json
Action Input: Olivia Wilde boyfriend[{'url': 'https://people.com/music/harry-styles-olivia-wilde-relationship-timeline/', 'content': 'August 24, 2022: Olivia Wilde says she and Harry Styles are protective of their relationship  Harry Styles and Olivia Wilde\'s Relationship Timeline  Styles and Olivia Wilde\'s relationship.  April 2022: Harry Styles reveals he\'s "really happy" amid his relationship with Olivia WildeNovember 18, 2022: Olivia Wilde and Harry Styles are "taking a break". After nearly two years together, Wilde and Styles are pressing pause on their romance. Multiple sources confirmed exclusively ...'}]I should use calculator to calculate the current age raised to the 0.23 power.
Action: calculator
Action Input: 23^0.23

> Entering new LLMMathChain chain...
23^0.23

> Entering new LLMChain chain..

{'input': "Who is Olivia Wilde's boyfriend? What is his current age raised to the 0.23 power?",
 'output': '2.0568252837687546'}

In [123]:
agent_executor.invoke({"input": "How much does a flight cost from Bangalore to Hyderabad? Multiply?"})



> Entering new AgentExecutor chain...
 We need to find the cost of a flight from Bangalore to Hyderabad and then multiply it by a certain number.
Action: tavily_search_results_json
Action Input: "flight cost Bangalore to Hyderabad"[{'url': 'https://www.easemytrip.com/flights/bangalore-blr-to-hyderabad-hyd/', 'content': 'Bangalore to Hyderabad Flights Details  Bangalore to Hyderabad Flight Information  HYD 09:00 Hyderabad Rajiv Gandhi International Airport Wed, 17 Jan 2024 Find the Lowest Airfare Bangalore to Hyderabad  Best Flight From Bangalore To Hyderabad BLR 07:25 Bangalore Kempegowda International Airport Wed , 17 Jan 2024 HYDReturn Date Book a round trip to save more TRAVELLER & CLASS Traveller (s) Economy Defence Forces Students Senior Citizens Doctors & Nurses Home Flights Bangalore To Hyderabad Flights January ₹12807 February ₹2972 March ₹2657 April ₹2775 May ₹2775 June ₹2775 July ₹2775 August'}] We need to select the appropriate flight cost from the search results.
Action: 

{'input': 'How much does a flight cost from Bangalore to Hyderabad? Multiply?',
 'output': 'The flight cost from Bangalore to Hyderabad is 12807 rupees and when multiplied by 2, the total cost would be 25614 rupees.'}